In [2]:
'''
this is about chroma

chroma is a light weight open source vector database, that is specially friendly for local deployment and small to medium scale production needs


Chroma is an open-source vector database designed for embedding storage and similarity search, commonly used in RAG systems.


At the top level, Chroma organizes data into collections, each representing a logical dataset.
Within a collection are documents (raw text or data units) linked to embeddings.

Each document has an embedding vector plus optional metadata for filtering.


Under the hood, Chroma manages indexes and persistent storage to enable fast, scalable retrieval.
'''

'\nthis is about chroma \n\nchroma is a light weight open source vector database, that is specially friendly for local deployment and small to medium scale production needs\n\n'

In [4]:
!pip install chromadb langchain_openai pypdf tiktoken openai langchain-community langchain

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = " "

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [12]:
from langchain_core.documents import Document


# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [13]:
docs = [doc1, doc2, doc3, doc4, doc5]


In [15]:
vectorstore = Chroma(
    persist_directory="chromadb",#here the name of the dir i want my database to be in
    embedding_function=OpenAIEmbeddings(),# the embedding model i am using
    collection_name="players", # the name of collection i want to put . collection is like a table in sql
)


In [16]:
vectorstore.add_documents(docs) # to add my data in the collection

['ab9cdce0-2cc0-4d6b-b8d8-5d8bad5337a6',
 '206a34df-da86-46fa-9308-e1a202affa02',
 'fd0493ab-2880-4028-b2a8-49706b7ad9b5',
 'a762b13c-9cc8-4c79-a251-41cd6c9354fd',
 'd2a5b1d7-9925-4f87-9e31-b030844642ab']

In [17]:
vectorstore.get(include=["embeddings","documents","metadatas"])

{'ids': ['ab9cdce0-2cc0-4d6b-b8d8-5d8bad5337a6',
  '206a34df-da86-46fa-9308-e1a202affa02',
  'fd0493ab-2880-4028-b2a8-49706b7ad9b5',
  'a762b13c-9cc8-4c79-a251-41cd6c9354fd',
  'd2a5b1d7-9925-4f87-9e31-b030844642ab'],
 'embeddings': array([[-0.00210453, -0.00214285,  0.0268    , ..., -0.01707893,
         -0.00366616,  0.01357884],
        [-0.00268021, -0.00010323,  0.02815653, ..., -0.01501936,
          0.00590092, -0.01164922],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [19]:
# search documents - in document searching the 2 nearest vector from the quesry vector
vectorstore.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='a762b13c-9cc8-4c79-a251-41cd6c9354fd', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='d2a5b1d7-9925-4f87-9e31-b030844642ab', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [20]:
# here the object is same just the simlarity score is also given

vectorstore.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='a762b13c-9cc8-4c79-a251-41cd6c9354fd', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.35445845127105713),
 (Document(id='d2a5b1d7-9925-4f87-9e31-b030844642ab', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.4085893929004669)]

In [21]:
# meta-data filtering
vectorstore.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

# here the ouput will be given on the bases of the filter team so all vectors of team chennai super super kings will be shown

[(Document(id='fd0493ab-2880-4028-b2a8-49706b7ad9b5', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.6488258242607117),
 (Document(id='d2a5b1d7-9925-4f87-9e31-b030844642ab', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.6566494703292847)]

In [24]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vectorstore.update_document(document_id='ab9cdce0-2cc0-4d6b-b8d8-5d8bad5337a6', document=updated_doc1)
# here the the id given in argument will anchor the doc that needs to be updated with updated document1

In [25]:
# view documents to seee the update
vectorstore.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['ab9cdce0-2cc0-4d6b-b8d8-5d8bad5337a6',
  '206a34df-da86-46fa-9308-e1a202affa02',
  'fd0493ab-2880-4028-b2a8-49706b7ad9b5',
  'a762b13c-9cc8-4c79-a251-41cd6c9354fd',
  'd2a5b1d7-9925-4f87-9e31-b030844642ab'],
 'embeddings': array([[-0.00544442, -0.01907989,  0.00706373, ..., -0.01627786,
         -0.00032134,  0.00724619],
        [-0.00268021, -0.00010323,  0.02815653, ..., -0.01501936,
          0.00590092, -0.01164922],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a sin

In [26]:
# delete document
vectorstore.delete(ids=['206a34df-da86-46fa-9308-e1a202affa02'])

In [28]:
# view documents
vectorstore.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['ab9cdce0-2cc0-4d6b-b8d8-5d8bad5337a6',
  'fd0493ab-2880-4028-b2a8-49706b7ad9b5',
  'a762b13c-9cc8-4c79-a251-41cd6c9354fd',
  'd2a5b1d7-9925-4f87-9e31-b030844642ab'],
 'embeddings': array([[-0.00544442, -0.01907989,  0.00706373, ..., -0.01627786,
         -0.00032134,  0.00724619],
        [ 0.00092799, -0.00476   ,  0.0124662 , ..., -0.01731381,
          0.00075886,  0.00296567],
        [-0.02714536,  0.00885395,  0.02699314, ..., -0.02592762,
          0.00900617, -0.01999116],
        [-0.01810451,  0.01281202,  0.0347942 , ..., -0.03034012,
         -0.00595078,  0.00521716]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to